In [ ]:
#@title imports
%pip install -q einops
import tensorflow as tf
from tensorflow.keras import layers, losses
from einops import *
from einops.layers.tensorflow import EinMix as Mix

path = '../input/intel-image-classification'

In [ ]:
#@title contrastive loss
class ContrastiveLoss(losses.Loss):
  def __init__(self):
    super().__init__()

  def call(self, p, z):
    p = tf.math.l2_normalize(p, axis=1)
    z = tf.math.l2_normalize(z, axis=1)
    return - reduce(einsum(p, z, 'b d, b d -> b'), 'b -> 1', 'mean')

In [ ]:
#@title backbone

#@title convolution
class Conv(layers.Layer):
    def __init__(self, filters, kernel, *args, **kwargs):
        super().__init__(*args)
        self.conv = layers.Conv2D(filters, kernel, **kwargs, padding='same', use_bias=False)
        self.ln = layers.BatchNormalization()
        self.selu = layers.Activation('relu')
        
    def call(self, inputs, training=False):
        inputs = self.conv(inputs, training=training)
        inputs = self.ln(inputs, training=training)
        inputs = self.selu(inputs)
        
        return inputs

#@title residual connection
class ResidualConnection(layers.Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = Conv(filters, 3) 
        self.conv2 = Conv(filters*2, 3) # double the filters
    
    def call(self, inputs, training=False):
        residual = inputs
        inputs = self.conv1(inputs, training=training)
        inputs = self.conv2(inputs, training=training)
        return inputs + residual

#@title backbone
class convnet(tf.keras.Model):
    def __init__(self, classes, filters):
        super().__init__()
        self.conv_1 = Conv(filters*2, 1, name='conv1')
        self.res_1 = ResidualConnection(filters, name='res1')
        self.conv_2 = Conv(filters*4, 1, name='conv2')
        self.res_2 = ResidualConnection(filters*2, name='res2') 
        self.conv_3 = Conv(filters*2, 1, name='conv3')
        self.globalpool = layers.GlobalMaxPooling2D(name='globalpool')
        self.fc = layers.Dense(classes, activation='softmax', name='fc')
        self.maxpool = layers.MaxPooling2D()
        
    def call(self, inputs, training=False):
        inputs = self.maxpool(self.conv_1(inputs, training=training))
        inputs = self.res_1(inputs, training=training)
        inputs = self.maxpool(self.conv_2(inputs, training=training))
        inputs = self.res_2(inputs, training=training)
        inputs = self.maxpool(self.conv_3(inputs, training=training))
        inputs = self.globalpool(inputs)
        output = self.fc(inputs, training=training)

        return output

In [ ]:
#@title projector
class Projector(layers.Layer):
  def __init__(self, dim):
    super().__init__()
    self.dim = dim
  
  def build(self, shape):
    self.mlp = tf.keras.Sequential([
        Mix('b i -> b j', weight_shape='i j', i=shape[1], j=shape[1]),
        layers.BatchNormalization(),
        layers.ReLU(),
        Mix('b i -> b j', weight_shape='i j', i=shape[1], j=shape[1]),
        layers.BatchNormalization(),
        layers.ReLU(),
        Mix('b i -> b d', weight_shape='i d', i=shape[1], d=self.dim),  
    ], name='projector')

  def call(self, inputs, training=False):
    return self.mlp(inputs, training=training)      

In [ ]:
#@title predictor
class Predictor(layers.Layer):
  def __init__(self, pred_dim):
    super().__init__()
    self.pred_dim = pred_dim
  
  def build(self, shape):
    self.mlp = tf.keras.Sequential([
        Mix('b i -> b d', weight_shape='i d', i=shape[1], d=self.pred_dim),
        layers.BatchNormalization(),
        layers.ReLU(),
        Mix('b d -> b i', weight_shape='d i', d=self.pred_dim, i=shape[1]),
    ], name='predictor')

  def call(self, inputs, training=False):
    return self.mlp(inputs, training=training)  

In [ ]:
#@title simsiam
class SimSiam(tf.keras.Model):
  def __init__(self, dim, pred_dim):
    super().__init__()
    self.encoder = convnet(dim, 64)
    self.encoder.fc = Projector(dim)
    self.predictor = Predictor(dim)
    self.loss_tracker = tf.keras.metrics.Mean(name="loss")

  def call(self, inputs, training=False):
    x1, x2 = inputs[0][0], inputs[0][1]
    z1 = self.encoder(x1, training=training) # n x d
    z2 = self.encoder(x2, training=training) # n x d
    p1 = self.predictor(z1, training=training)
    p2 = self.predictor(z2, training=training)
    return p1, p2, tf.stop_gradient(z1), tf.stop_gradient(z2)

  def train_step(self, inputs):
    with tf.GradientTape() as tape:
      p1, p2, z1, z2 = self(inputs, training=True)
      loss = self.compiled_loss(p1, z2)*0.5 + self.compiled_loss(p2, z1)*0.5
      trainable_vars = self.trainable_variables
      gradients = tape.gradient(loss, trainable_vars)
      self.optimizer.apply_gradients(zip(gradients, trainable_vars))
      self.loss_tracker.update_state(loss)
      return {"loss": self.loss_tracker.result()}

  @property
  def metrics(self):
    return [self.loss_tracker]

In [ ]:
#@title data preparation
AUTOTUNE = tf.data.AUTOTUNE
train_ds = tf.keras.utils.image_dataset_from_directory(path + "/seg_train/seg_train", image_size=(64, 64), label_mode='int', batch_size=8)
test_ds = tf.keras.utils.image_dataset_from_directory(path + "/seg_test/seg_test", label_mode='int', image_size=(64, 64), batch_size=1)

def augment(images, labels):
    x1, x2 = tf.image.random_flip_left_right(images), tf.image.random_flip_left_right(images)
    x1, x2 = tf.image.random_flip_up_down(x1), tf.image.random_flip_up_down(x2)
    x1, x2 = tf.image.rot90(x1), tf.image.rot90(x2)
    x1, x2 = tf.image.random_brightness(x1, max_delta=0.8), tf.image.random_brightness(x2, max_delta=0.8)
    x1, x2 = tf.image.random_contrast(x1, lower=1-0.8, upper=1+0.8), tf.image.random_contrast(x2, lower=1-0.8, upper=1+0.8)
    x1, x2 = tf.image.random_saturation(x1, lower=1-0.8, upper=1+0.8), tf.image.random_saturation(x2, lower=1-0.8, upper=1+0.8)
    x1, x2 = tf.image.random_hue(x1, max_delta=0.2), tf.image.random_hue(x2, max_delta=0.2)
    return (x1/255, x2/255), labels

train_ds = (
    train_ds
    .cache()
    .shuffle(1024, seed=0)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .prefetch(AUTOTUNE)
)

test_ds = test_ds.map(lambda x, y: (x/255.0, y))
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
#@title training - contrastive learning
simsiam = SimSiam(dim=2048, pred_dim=512)
epochs = 100

#@title loss and optimizer
loss = ContrastiveLoss()
scheduler = tf.keras.optimizers.schedules.CosineDecay(0.05, 500)
optimizer = tf.keras.optimizers.SGD(learning_rate=scheduler, momentum=0.9, decay=1e-04)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15)

#@title compile and train
simsiam.compile(optimizer=optimizer, loss=loss)
simsiam.fit(train_ds, epochs=epochs, callbacks=[early_stop])

In [ ]:
#@title testing - image classification
train_ds = tf.keras.utils.image_dataset_from_directory(path + "/seg_train/seg_train", image_size=(64, 64), label_mode='int', batch_size=8)
train_ds = train_ds.cache().shuffle(1024, seed=0).map(lambda x, y: (x/255.0, y), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

classes = 6
convnet = simsiam.encoder
convnet.fc = layers.Dense(classes, activation='softmax', name='fc')
scheduler = tf.keras.optimizers.schedules.CosineDecay(0.01, 500)
optimizer = tf.keras.optimizers.SGD(learning_rate=scheduler, momentum=0.9, decay=1e-04)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)


convnet.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

convnet.fit(train_ds, validation_data=test_ds, epochs=50, callbacks=[early_stop])